# Import thư viện


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, SimpleRNN, GRU, Bidirectional
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
from keras.callbacks import ModelCheckpoint

# Đọc dữ liệu

In [ ]:
# Đọc dữ liệu vào DataFrame
df_t_2 = pd.read_csv(r'E:\CoChau\Fair_2024_LSTM_GA-LSTM_BiLSTM_GA-BiLSTM\Code\data\data_t_2\TongMua_KaNak_AnKhe_t-2.csv')
df_t_4 = pd.read_csv(r'E:\CoChau\Fair_2024_LSTM_GA-LSTM_BiLSTM_GA-BiLSTM\Code\data\data_t_4\TongMua_KaNak_AnKhe_t-4.csv')
df_t_6 = pd.read_csv(r'E:\CoChau\Fair_2024_LSTM_GA-LSTM_BiLSTM_GA-BiLSTM\Code\data\data_t_6\TongMua_KaNak_AnKhe_t-6.csv')

# Xử lý dữ liệu bị thiếu
df_t_2 = df_t_2.dropna()
df_t_4 = df_t_4.dropna()
df_t_6 = df_t_6.dropna()

# Tạo dữ liệu huấn luyện

In [ ]:
def create_dataset(X, y, n_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - n_steps):
        Xs.append(X[i:i+n_steps])
        ys.append(y[i+n_steps])
    return np.array(Xs), np.array(ys)

# Chuẩn hóa và chia bộ dữ liệu cho mô hình

In [ ]:
# Chuẩn hóa dữ liệu
def scaler(df, time_steps):
    scaler = MinMaxScaler()
    X = df[['Tổng lưu lượng xả (m³/s)[Thực tế]', 'Lưu lượng đến hồ (m³/s)', 'Sum_Mua']]
    y = df[['Lưu lượng đến hồ (m³/s)']]
    X = scaler.fit_transform(X)
    y = scaler.fit_transform(y)

    # Tạo nhãn cho dữ liệu
    X_data, y_data = create_dataset(X, y, time_steps)

    return scaler, X_data, y_data

In [ ]:
# Chuẩn hóa dữ liệu
def split_data(X_data, y_data, time_steps):
    # Chia dữ liệu thành tập huấn luyện, kiểm tra và test
    train_length = int(len(X_data) * 0.7)
    val_length = int((len(X_data) - train_length) / 2)

    X_train, y_train = X_data[:train_length], y_data[:train_length]
    X_val, y_val = X_data[train_length:train_length+val_length], y_data[train_length:train_length+val_length]
    X_test, y_test = X_data[-val_length:], y_data[-val_length:]

    # Reshape dữ liệu đầu vào
    X_train = X_train.reshape(X_train.shape[0], time_steps, X_train.shape[2])
    X_val = X_val.reshape(X_val.shape[0], time_steps, X_val.shape[2])
    X_test = X_test.reshape(X_test.shape[0], time_steps, X_test.shape[2])

    print("Kích thước tập huấn luyện:", len(X_train))
    print("Kích thước tập validation:", len(X_val))
    print("Kích thước tập kiểm tra:", len(X_test))

    return X_train, y_train, X_val, y_val, X_test, y_test

# Xây dựng các mô hình

In [ ]:
# Tạo mô hình RNN
def model_rnn(input_shape):
    model = Sequential([
        SimpleRNN(32, input_shape=input_shape, activation='tanh', return_sequences=False),
        Dropout(0.5),
        Dense(1, activation='sigmoid')])
    return model

# Tạo mô hình LSTM
def model_lstm(input_shape):
    model = Sequential()
    model.add(LSTM(32, input_shape=input_shape, activation='tanh', return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Tạo mô hình BiLSTM
def model_bilstm(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(32, input_shape=input_shape, activation='tanh', return_sequences=False)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Tạo mô hình GRU
def model_gru(input_shape):
    model = Sequential()
    model.add(GRU(32, input_shape=input_shape, activation='tanh', return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, epochs, batch_size, save_best_model):
    # Compile mô hình
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='mean_squared_error')

    # Lưu lại model tốt nhất
    model_checkpoint = ModelCheckpoint(save_best_model, save_best_only=True, monitor='val_loss', mode='min', verbose=1)

    # Train the model
    history = model.fit(X_train, y_train, 
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(X_val, y_val),
                        verbose=1,
                        callbacks=model_checkpoint)
    
    return model, history

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_model(history):
    # Trích xuất giá trị loss trên tập huấn luyện và kiểm tra
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Vẽ biểu đồ
    plt.plot(train_loss, marker='.', linestyle='-')
    plt.plot(val_loss, marker='', linestyle='-')
    plt.xlabel('Epoch')
    plt.ylabel('Validation Loss')
    plt.title('Validation Loss over Epochs')
    plt.yscale('log')  # Scale y-axis to logarithmic scale
    plt.show()

# Dự đoán trên tập kiểm tra và tính toán các chỉ số

In [ ]:
def evaluate(model, X_test, y_test):
    # Dự đoán trên tập validation
    y_pred = model.predict(X_test)

    # Calculate evaluation metrics
    mse_test = mean_squared_error(y_test, y_pred)
    mae_test = mean_absolute_error(y_test, y_pred)
    r2_test = r2_score(y_test, y_pred)
    rmse_test = sqrt(mse_test)
    rmae_test = sqrt(mae_test)

    # Print evaluation results
    print("Mean Squared Error (MSE) on test set:", mse_test)
    print("Mean Absolute Error (MAE) on test set:", mae_test)
    print("R-squared (R2) on test set:", r2_test)
    print("Root Mean Squared Error (RMSE) on test set:", rmse_test)
    print("Root Mean Absolute Error (RMAE) on test set:", rmae_test)

    return y_pred, r2_test

# Hiển thị biểu đồ kết quả

In [ ]:
# Khôi phục dữ liệu sau chuẩn hóa
def inverse_transform_data(scaler, y_pred, y_test):
    y_pred_predict = scaler.inverse_transform(y_pred)
    y_pred_predict = pd.DataFrame(y_pred_predict)

    y_test = scaler.inverse_transform(y_test)
    y_test = pd.DataFrame(y_test)

    return y_pred_predict, y_test

In [ ]:
def plot_data_goc(df):
    # Vẽ biểu đồ dữ liệu gốc
    plt.figure(figsize=(15, 6))
    plt.plot(df['thoi_gian_KaNak'], df['Tổng lưu lượng xả (m³/s)[Thực tế]'], label='Tổng lưu lượng xả')
    plt.plot(df['thoi_gian_KaNak'], df['Lưu lượng đến hồ (m³/s)'], label='Lưu lượng đến hồ')
    plt.plot(df['thoi_gian_KaNak'], df['Sum_Mua'], label='Tổng lượng mưa')
    plt.xlabel('Thời gian')
    plt.ylabel('Lưu lượng (m³/s)')
    plt.title('Biểu đồ lưu lượng')
    plt.legend()
    plt.show()

def plot_data_predict(y_test, y_pred_predict):
    # Biểu đồ giá trị dự đoán và thực tế trên tập test
    plt.figure(figsize=(15, 6))
    plt.plot(y_test, label='Lưu lượng đến hồ thực tế')
    plt.plot(y_pred_predict, label='Lưu lượng đến hồ dự đoán')
    plt.xlabel('Thời gian')
    plt.ylabel('Lưu lượng đến hồ (m³/s)')
    plt.title('So sánh giá trị dự đoán và giá trị thực tế trên tập kiểm tra')
    plt.legend()
    plt.show()

def plot_bat_thuong_test(y_test, y_pred_predict):
    # Biểu đồ giá trị dự đoán và giá trị thực tế ở điểm bất thường trên tập test
    plt.figure(figsize=(15, 6))
    plt.plot(y_test, label='Lưu lượng đến hồ thực tế')
    plt.plot(y_pred_predict, label='Lưu lượng đến hồ dự đoán')
    plt.xlabel('Thời gian')
    plt.ylabel('Lưu lượng đến hồ (m³/s)')
    plt.title('So sánh giá trị dự đoán và giá trị thực tế trên tập kiểm tra')
    plt.legend()

    # Chỉ hiển thị một đoạn nhỏ của biểu đồ
    start_index = 0
    end_index = 1000
    plt.xlim(start_index, end_index)
    plt.show() 

def plot_data_chi_dinh(scaler, model, X_train, y_train):
    X_data_slipt = X_train[15290:17980]
    y_data_slipt = y_train[15290:17980]

    # Dự đoán trên đoạn dữ liệu chỉ định
    y_data_slipt_predict = model.predict(X_data_slipt)

    # Khôi phục dữ liệu sau chuẩn hóa
    y_data_slipt_predict = scaler.inverse_transform(y_data_slipt_predict)
    y_data_slipt_predict = pd.DataFrame(y_data_slipt_predict)
    y_data_slipt = scaler.inverse_transform(y_data_slipt)
    y_data_slipt = pd.DataFrame(y_data_slipt)

    # Biểu đồ giá trị dự đoán và giá trị thực tế trên đoạn dữ liệu gốc bất kỳ
    plt.figure(figsize=(15, 6))
    plt.plot(y_data_slipt, label='Lưu lượng đến hồ thực tế')
    plt.plot(y_data_slipt_predict, label='Lưu lượng đến hồ dự đoán')
    plt.xlabel('Thời gian')
    plt.ylabel('Lưu lượng đến hồ (m³/s)')
    plt.title('So sánh giá trị dự đoán và giá trị thực tế trên tập kiểm tra')
    plt.legend()
    plt.show()

# Chạy mô hình

In [ ]:
# Các tham số cho mô hình
time_steps = 24 * 3
epochs = 5
batch_size = 24

# Chuẩn hóa dữ liệu
scaler_t_2, X_data_t_2, y_data_t_2 = scaler(df_t_2, time_steps)
scaler_t_4, X_data_t_4, y_data_t_4 = scaler(df_t_4, time_steps)
scaler_t_6, X_data_t_6, y_data_t_6 = scaler(df_t_6, time_steps)

# Chia dữ liệu
print("Data cách 2 giờ")
X_train_t_2, y_train_t_2, X_val_t_2, y_val_t_2, X_test_t_2, y_test_t_2 = split_data(X_data_t_2, y_data_t_2, time_steps)
print()
print("Data cách 4 giờ")
X_train_t_4, y_train_t_4, X_val_t_4, y_val_t_4, X_test_t_4, y_test_t_4 = split_data(X_data_t_4, y_data_t_4, time_steps)
print()
print("Data cách 6 giờ")
X_train_t_6, y_train_t_6, X_val_t_6, y_val_t_6, X_test_t_6, y_test_t_6 = split_data(X_data_t_6, y_data_t_6, time_steps)

In [ ]:
# Đầu vào cho các mô hình
input_shape = (X_train_t_2.shape[1], X_train_t_2.shape[2])

# Khởi tạo các mô hình
rnn_model = model_rnn(input_shape)
lstm_model = model_lstm(input_shape)
bilstm_model = model_bilstm(input_shape)
gru_model = model_gru(input_shape)
total_model = [rnn_model, lstm_model, bilstm_model, gru_model]

In [ ]:
r2_t_2_model = []
r2_t_4_model = []
r2_t_6_model = []

for i in range(len(total_model)):
    if i == 0:
        name_model = "RNN"
    elif i == 1:
        name_model = "LSTM"
    elif i == 2:
        name_model = "BiLSTM"
    elif i == 3:
        name_model = "GRU"
    
    # Lưu mô hình tốt nhất
    save_best_model_t_2 = f'./best_model_v2/Model_{name_model}_t-2.hdf5'
    save_best_model_t_4 = f'./best_model_v2/Model_{name_model}_t-4.hdf5'
    save_best_model_t_6 = f'./best_model_v2/Model_{name_model}_t-6.hdf5'
    
    # Gọi mô hình
    model = total_model[i]
    print(f"MÔ HÌNH {name_model}")
    print()

    # Training mô hình
    print(f"Training mô hình {name_model} với data cách 2 giờ")
    model_t_2, history_t_2 = train_model(model, X_train_t_2, y_train_t_2, X_val_t_2, y_val_t_2, 
                                        epochs, batch_size, save_best_model_t_2)
    print(f"Training mô hình {name_model} với data cách 4 giờ")
    model_t_4, history_t_4 = train_model(model, X_train_t_4, y_train_t_4, X_val_t_4, y_val_t_4, 
                                        epochs, batch_size, save_best_model_t_4)
    print(f"Training mô hình {name_model} với data cách 6 giờ")
    model_t_6, history_t_6 = train_model(model, X_train_t_6, y_train_t_6, X_val_t_6, y_val_t_6, 
                                        epochs, batch_size, save_best_model_t_6)
    print()

    # Vẽ biểu đồ train_loss và val_loss
    # Đánh giá mô hình trên tập test
    print(f"Train loss và Val loss của mô hình {name_model} với data cách 2 giờ")
    plot_loss_model(history_t_2)
    print(f"Đánh giá mô hình {name_model} với data cách 2 giờ")
    y_pred_t_2, r2_t_2 = evaluate(model_t_2, X_test_t_2, y_test_t_2)
    r2_t_2_model.append(r2_t_2)
    print()

    print(f"Train loss và Val loss của mô hình {name_model} với data cách 4 giờ")
    plot_loss_model(history_t_4)
    print(f"Đánh giá mô hình {name_model} với data cách 4 giờ")
    y_pred_t_4, r2_t_4 = evaluate(model_t_4, X_test_t_4, y_test_t_4)
    r2_t_4_model.append(r2_t_4)
    print()

    print(f"Train loss và Val loss của mô hình {name_model} với data cách 6 giờ")
    plot_loss_model(history_t_6)
    print(f"Đánh giá mô hình {name_model} với data cách 6 giờ")
    y_pred_t_6, r2_t_6 = evaluate(model_t_6, X_test_t_6, y_test_t_6)
    r2_t_6_model.append(r2_t_6)
    print()

    # Chuẩn hóa lại dữ liệu sau để vẽ mô hình
    y_pred_predict_t_2, y_test_t_2 = inverse_transform_data(scaler_t_2, y_pred_t_2, y_test_t_2)
    y_pred_predict_t_4, y_test_t_4 = inverse_transform_data(scaler_t_4, y_pred_t_4, y_test_t_4)
    y_pred_predict_t_6, y_test_t_6 = inverse_transform_data(scaler_t_6, y_pred_t_6, y_test_t_6)

    # Biểu đồ giá trị dự đoán và thực tế trên tập test
    print(f"Dữ liệu dự đoán trên tập test của mô hình {name_model} với data cách 2 giờ")
    plot_data_predict(y_test_t_2, y_pred_predict_t_2)
    print(f"Dữ liệu dự đoán trên tập test của mô hình {name_model} với data cách 4 giờ")
    plot_data_predict(y_test_t_4, y_pred_predict_t_4)
    print(f"Dữ liệu dự đoán trên tập test của mô hình {name_model} với data cách 6 giờ")
    plot_data_predict(y_test_t_6, y_pred_predict_t_6)
    print()

    # Biểu đồ giá trị dự đoán và giá trị thực tế ở điểm bất thường trên tập test
    print(f"Dự đoán dữ liệu bất thường trên tập test của mô hình {name_model} với dữ liệu cách 2 giờ")
    plot_bat_thuong_test(y_test_t_2, y_pred_predict_t_2)
    print(f"Dự đoán dữ liệu bất thường trên tập test của mô hình {name_model} với dữ liệu cách 4 giờ")
    plot_bat_thuong_test(y_test_t_4, y_pred_predict_t_4)
    print(f"Dự đoán dữ liệu bất thường trên tập test của mô hình {name_model} với dữ liệu cách 6 giờ")
    plot_bat_thuong_test(y_test_t_6, y_pred_predict_t_6)
    print()

    # Vẽ biểu đồ tại điểm bất thường trên dữ liệu gốc
    print(f"Điểm bất thường trên dữ liệu gốc của mô hình {name_model} cách 2 giờ")
    plot_data_chi_dinh(scaler_t_2, model_t_2, X_train_t_2, y_train_t_2)
    print(f"Điểm bất thường trên dữ liệu gốc của mô hình {name_model} cách 4 giờ")
    plot_data_chi_dinh(scaler_t_4, model_t_4, X_train_t_4, y_train_t_4)
    print(f"Điểm bất thường trên dữ liệu gốc của mô hình {name_model} cách 6 giờ")
    plot_data_chi_dinh(scaler_t_6, model_t_6, X_train_t_6, y_train_t_6)
    print()

### So sánh tìm mô hình tốt nhất

In [ ]:
def best_model_r2(r2_model):
    max_value = max(r2_model)

    # Tìm vị trí của giá trị lớn nhất
    max_index = r2_model.index(max_value)

    if max_index == 0:
        name_model = "RNN"
    elif max_index == 1:
        name_model = "LSTM"
    elif max_index == 2:
        name_model = "BiLSTM"
    elif max_index == 3:
        name_model = "GRU"

    print(f"R2 tốt nhất là: {max_value}")
    print(f"Mô hình tốt nhất là: {name_model}")

In [ ]:
best_model_r2(r2_t_2_model)
best_model_r2(r2_t_4_model)
best_model_r2(r2_t_6_model)